<a href="https://colab.research.google.com/github/dercodeKoenig/Turtle-grafik/blob/main/gan_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


import os
import matplotlib.pyplot as plt
import tensorflow as tf
import glob
import numpy as np
import os
from tensorflow.keras import layers
import time
import cv2
from IPython import display

t0= time.time()

train_images_cartoon = np.load("npfilet.npy")
train_images_cartoon = tf.keras.layers.experimental.preprocessing.RandomZoom((0,0.4),fill_mode='constant',fill_value=1)(train_images_cartoon)
train_images_cartoon = tf.keras.layers.experimental.preprocessing.RandomTranslation((-0.2,0.2), (-0.2,0.2),fill_mode='constant',fill_value=1)(train_images_cartoon)

train_images_cartoon1=train_images_cartoon[0:int(1*train_images_cartoon.shape[0]/4)]
train_images_cartoon2=train_images_cartoon[int(1*train_images_cartoon.shape[0]/4):int(2*train_images_cartoon.shape[0]/4)]
train_images_cartoon3=train_images_cartoon[int(2*train_images_cartoon.shape[0]/4):int(3*train_images_cartoon.shape[0]/4)]
train_images_cartoon4=train_images_cartoon[int(3*train_images_cartoon.shape[0]/4):int(4*train_images_cartoon.shape[0]/4)]
del train_images_cartoon

small_images_cartoon = np.load("npfiles.npy")
small_images_cartoon = tf.keras.layers.experimental.preprocessing.RandomZoom((0,0.4),fill_mode='constant',fill_value=1)(small_images_cartoon)
small_images_cartoon = tf.keras.layers.experimental.preprocessing.RandomTranslation((-0.2,0.2), (-0.2,0.2),fill_mode='constant',fill_value=1)(small_images_cartoon)
test_imgs = small_images_cartoon[0:20]

small_images_cartoon1=small_images_cartoon[0:int(1*small_images_cartoon.shape[0]/4)]
small_images_cartoon2=small_images_cartoon[int(1*small_images_cartoon.shape[0]/4):int(2*small_images_cartoon.shape[0]/4)]
small_images_cartoon3=small_images_cartoon[int(2*small_images_cartoon.shape[0]/4):int(3*small_images_cartoon.shape[0]/4)]
small_images_cartoon4=small_images_cartoon[int(3*small_images_cartoon.shape[0]/4):int(4*small_images_cartoon.shape[0]/4)]
del small_images_cartoon


BATCH_SIZE_PER_REPLICA = 6
GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * 8 #8TPUs

try:
  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
  tf.config.experimental_connect_to_cluster(resolver)
  tf.tpu.experimental.initialize_tpu_system(resolver)
  strategy = tf.distribute.TPUStrategy(resolver)
except:
  strategy = tf.distribute.MirroredStrategy()

train_images_cartoon1 = tf.data.Dataset.from_tensor_slices((train_images_cartoon1)).batch(GLOBAL_BATCH_SIZE)  
train_images_cartoon2 = tf.data.Dataset.from_tensor_slices((train_images_cartoon2)).batch(GLOBAL_BATCH_SIZE) 
train_images_cartoon3 = tf.data.Dataset.from_tensor_slices((train_images_cartoon3)).batch(GLOBAL_BATCH_SIZE) 
train_images_cartoon4 = tf.data.Dataset.from_tensor_slices((train_images_cartoon4)).batch(GLOBAL_BATCH_SIZE) 
train_images_cartoon1 = strategy.experimental_distribute_dataset(train_images_cartoon1)
train_images_cartoon2 = strategy.experimental_distribute_dataset(train_images_cartoon2)
train_images_cartoon3 = strategy.experimental_distribute_dataset(train_images_cartoon3)
train_images_cartoon4 = strategy.experimental_distribute_dataset(train_images_cartoon4)

small_images_cartoon1 = tf.data.Dataset.from_tensor_slices((small_images_cartoon1)).batch(GLOBAL_BATCH_SIZE)  
small_images_cartoon2 = tf.data.Dataset.from_tensor_slices((small_images_cartoon2)).batch(GLOBAL_BATCH_SIZE) 
small_images_cartoon3 = tf.data.Dataset.from_tensor_slices((small_images_cartoon3)).batch(GLOBAL_BATCH_SIZE) 
small_images_cartoon4 = tf.data.Dataset.from_tensor_slices((small_images_cartoon4)).batch(GLOBAL_BATCH_SIZE) 
small_images_cartoon1 = strategy.experimental_distribute_dataset(small_images_cartoon1)
small_images_cartoon2 = strategy.experimental_distribute_dataset(small_images_cartoon2)
small_images_cartoon3 = strategy.experimental_distribute_dataset(small_images_cartoon3)
small_images_cartoon4 = strategy.experimental_distribute_dataset(small_images_cartoon4)


def make_generator_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(64,64,3)))
    
    model.add(layers.Conv2D(128, 5))
    model.add(layers.Dropout(0.2))
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2D(64, 5))
    model.add(layers.Dropout(0.2))
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2D(32, 5))
    model.add(layers.Dropout(0.2))
    model.add(layers.LeakyReLU())
    
    model.add(layers.Flatten())
    model.add(layers.Dense(8*8*32, use_bias=False))
    model.add(layers.Dropout(0.2))
    model.add(layers.LeakyReLU())
    
    model.add(layers.Reshape((8,8,32)))


    model.add(layers.Conv2D(128, 5,padding="same", use_bias=False))
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2D(128, 5,padding="same", use_bias=False))
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2D(128, 5,padding="same", use_bias=False))
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2D(128, 5,padding="same", use_bias=False))
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2D(128, 5,padding="same", use_bias=False))
    model.add(layers.LeakyReLU())



    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1),padding="same", use_bias=False))
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2D(128, 5,padding="same", use_bias=False))
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2D(128, 5,padding="same", use_bias=False))
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2D(128, 5,padding="same", use_bias=False))
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2D(128, 5,padding="same", use_bias=False))
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2D(128, 5,padding="same", use_bias=False))
    model.add(layers.LeakyReLU())



    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2),padding="same", use_bias=False))
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2D(128, 5,padding="same", use_bias=False))
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2D(128, 5,padding="same", use_bias=False))
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2D(128, 5,padding="same", use_bias=False))
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2D(128, 5,padding="same", use_bias=False))
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2D(128, 5,padding="same", use_bias=False))
    model.add(layers.LeakyReLU())



    model.add(layers.Conv2DTranspose(128, (5, 5),padding="same", strides=(2, 2), use_bias=False))
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2D(128, 5,padding="same", use_bias=False))
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2D(128, 5,padding="same", use_bias=False))
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2D(128, 5,padding="same", use_bias=False))
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2D(128, 5,padding="same", use_bias=False))
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2D(128, 5,padding="same", use_bias=False))
    model.add(layers.LeakyReLU())

    
    model.add(layers.Conv2DTranspose(128, (5, 5),padding="same", strides=(2, 2), use_bias=False))
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2D(128, 5,padding="same", use_bias=False))
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2D(128, 5,padding="same", use_bias=False))
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2D(128, 5,padding="same", use_bias=False))
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2D(128, 5,padding="same", use_bias=False))
    model.add(layers.LeakyReLU())


    model.add(layers.Conv2DTranspose(3,5,padding="same",strides=(1, 1), activation='tanh', use_bias=False))


    return model


def make_discriminator_model():
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Input(shape=(64,64,3)))

    model.add(layers.Conv2D(16, (5, 5), strides=(1, 1)))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(32, (5, 5), strides=(1, 1)))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(1, 1)))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(256, (5, 5), strides=(1, 1)))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(256, (5, 5), strides=(1, 1)))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(256, (5, 5), strides=(1, 1)))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(256, (5, 5), strides=(1, 1)))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(256, (3, 3), strides=(1, 1)))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(256, (3, 3), strides=(1, 1)))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(256, (3, 3), strides=(1, 1)))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())


    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(128))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(1,activation="sigmoid"))

    return model

with strategy.scope():
    discriminator_cartoon = make_discriminator_model()
    generator = make_generator_model()

with strategy.scope():
    cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False,reduction=tf.keras.losses.Reduction.NONE)

    def discriminator_cartoon_loss(real_output, fake_output):
        real_loss = cross_entropy(tf.ones_like(real_output), real_output)
        fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
        total_loss = real_loss + fake_loss
        return  tf.nn.compute_average_loss(total_loss, global_batch_size=GLOBAL_BATCH_SIZE)


    def _gaussian_kernel(kernel_size, sigma, n_channels, dtype):
        x = tf.range(-kernel_size // 2 + 1, kernel_size // 2 + 1, dtype=dtype)
        g = tf.math.exp(-(tf.pow(x, 2) / (2 * tf.pow(tf.cast(sigma, dtype), 2))))
        g_norm2d = tf.pow(tf.reduce_sum(g), 2)
        g_kernel = tf.tensordot(g, g, axes=0) / g_norm2d
        g_kernel = tf.expand_dims(g_kernel, axis=-1)
        return tf.expand_dims(tf.tile(g_kernel, (1, 1, n_channels)), axis=-1)


    def apply_blur(img):
        blur = _gaussian_kernel(17, 30, 3, img.dtype)
        img = tf.nn.depthwise_conv2d(img, blur, [1,1,1,1], 'SAME')
        return img


    def generator_loss(fake_output,norm,genr):
        norm=apply_blur(norm)
        genr=apply_blur(genr)
        total_loss=tf.nn.compute_average_loss(((norm-genr)*(norm-genr)))*0.3+0.7*tf.nn.compute_average_loss((cross_entropy(tf.ones_like(fake_output), fake_output)))
        return  (total_loss)

    generator_optimizer = tf.keras.optimizers.Adam(0.000016)
    discriminator_cartoon_optimizer = tf.keras.optimizers.Adam(0.00002)

  
    def train_step_cartoon(cartoons,dataset_seed):

          with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_cartoon_tape :
            
            generated_images = generator(dataset_seed, training=True)

            real_output_cartoon = discriminator_cartoon(cartoons, training=True)

            fake_output_cartoon = discriminator_cartoon(generated_images, training=True)

            gen_loss = generator_loss(fake_output_cartoon,dataset_seed,generated_images)
        
            disc_cartoon_loss = discriminator_cartoon_loss(real_output_cartoon, fake_output_cartoon)
      
          gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
          gradients_of_discriminator_cartoon = disc_cartoon_tape.gradient(disc_cartoon_loss, discriminator_cartoon.trainable_variables)
      
          generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
      
          discriminator_cartoon_optimizer.apply_gradients(zip(gradients_of_discriminator_cartoon, discriminator_cartoon.trainable_variables))
      
        
  
@tf.function
def train(dataset_seed,dataset_cartoon):
    itt = iter(dataset_cartoon)
    for seed_batch in dataset_seed:
        strategy.run(train_step_cartoon,args=(itt.get_next(),seed_batch,))

    


resume = True
if(resume):
    generator.load_weights('gen_weights.h5')
    discriminator_cartoon.load_weights('disc_weights.h5')



/content/drive/MyDrive
INFO:tensorflow:Initializing the TPU system: grpc://10.119.126.218:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.119.126.218:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
def generate_images():
  display.clear_output(wait=True)
  predictions = model(test_imgs,training=False)
  generated_images = np.array(predictions,dtype= 'float32')

  fig = plt.figure(figsize=(4, 10))

  for i in range(generated_images.shape[0]):
    img =cv2.cvtColor(generated_images[i],cv2.COLOR_BGR2RGB)
    img2 =cv2.cvtColor(test_imgs[i],cv2.COLOR_BGR2RGB)
    plt.subplot(4, 10, i+1)
    plt.imshow(img2)

    plt.subplot(4, 10, i+2)
    plt.imshow(img)

  plt.show()


while(True):
    generate_images(generator,test_img)    

    train(small_images_cartoon2,train_images_cartoon2)
    print("1")
    train(small_images_cartoon4,train_images_cartoon4)
    print("2")
    train(small_images_cartoon3,train_images_cartoon3)  
    print("3")
    train(small_images_cartoon1,train_images_cartoon1)
    print("4")
    
    train(small_images_cartoon2,train_images_cartoon1)
    print("5")
    train(small_images_cartoon4,train_images_cartoon2)
    print("6")
    train(small_images_cartoon3,train_images_cartoon4)  
    print("7")
    train(small_images_cartoon1,train_images_cartoon3)
    print("8")
    
    train(small_images_cartoon2,train_images_cartoon3)
    print("9")
    train(small_images_cartoon4,train_images_cartoon1)
    print("10")
    train(small_images_cartoon3,train_images_cartoon2)  
    print("11")
    train(small_images_cartoon1,train_images_cartoon4)
    print("12")
    
    train(small_images_cartoon2,train_images_cartoon4)
    print("13")
    train(small_images_cartoon4,train_images_cartoon3)
    print("14")
    train(small_images_cartoon3,train_images_cartoon1)  
    print("15")
    train(small_images_cartoon1,train_images_cartoon2)
    print("16")
    
    print("save chkp")
    generator.save_weights('gen_weights.h5', overwrite=True)
    discriminator_cartoon.save_weights('disc_weights.h5', overwrite=True)
